In [1]:
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

#### Neural Networks
使用`torch.nn`包来构建神经网络。
`nn`包依赖`autograd`包来定义模型并求导。一个`nn.Module`包含各个层和一个`forward(input)`方法，该方法返回`output`。

神经网络的典型训练过程如下：
1. 定义包含一些可学习的参数（或者叫权重）神经网络模型；
2. 在数据集上迭代；
3. 通过神经网络处理输入；
4. 计算损失（输入结果和正确值的差值大小）；
5. 将梯度反向传播回网络的参数；
6. 更新网络的参数，主要使用如下简单的更新原则：`weight = weight - learning_rate * gradient`

#### 定义网络

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 定义网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层1，输入通道1，输出通道6，卷积核5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 全连接层1
        self.fc2 = nn.Linear(120, 84)          # 全连接层2
        self.fc3 = nn.Linear(84, 10)           # 全连接层3

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # 卷积层1 + ReLU + 池化
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)    # 卷积层2 + ReLU + 池化
        x = x.view(-1, self.num_flat_features(x))  # 展平多维的卷积图成一维的向量
        x = F.relu(self.fc1(x))                     # 全连接层1 + ReLU
        x = F.relu(self.fc2(x))                     # 全连接层2 + ReLU
        x = self.fc3(x)                             # 全连接层3
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批量大小外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net() # 创建网络实例
print(net) # 打印网络结构

ModuleNotFoundError: No module named 'torch'

在模型中必须要定义`forward`函数，`backward`函数（用来计算梯度）会被`autograd`自动创建。可以在`forward`函数中使用任何针对`Tensor`的操作。

`net.parameters()`返回可被学习的参数（权重）列表和值

In [4]:
params = list(net.parameters()) # 获取网络的所有参数
print(len(params)) # 打印参数的数量
print(params[0].size()) # 打印第一个参数的尺寸（卷积层1的权重）

NameError: name 'net' is not defined

测试随机输入32*32。

注：这个网络（LeNet）期望的输入大小是32 * 32，如果使用MINIST数据集来训练这个网络，请把图片大小重新调整到32*32。

In [ ]:
input = torch.randn(1, 1, 32, 32) # 创建一个随机输入张量，尺寸为(1, 1, 32, 32)
out = net(input) # 前向传播
print(out) # 打印输出结果

将所有参数的梯度缓存清零，然后进行随机梯度的反向传播：

In [ ]:
net.zero_grad() # 将所有参数的梯度缓存清零
out.backward(torch.randn(1, 10)) # 随机梯度的反向传播